## 딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회

**※주의** : 반드시 본 파일을 이용하여 제출을 수행해야 하며, 파일의 이름은 `task.ipynb`로 유지되어야 합니다.

* #### 추론 실행 환경
    * `python 3.9` 환경
    * `CUDA 10.2`, `CUDA 11.8`, `CUDA 12.6`를 지원합니다.
    * 각 CUDA 환경에 미리 설치돼있는 torch 버전은 다음 표를 참고하세요.

<table>
  <thead>
    <tr>
      <th align="center">Python</th>
      <th align="center">CUDA</th>
      <th align="center">torch</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="center" style="vertical-align: middle;">3.8</td>
      <td align="center">10.2</td>
      <td align="center">1.6.0</td>
    </tr>
    <tr>
      <td align="center" style="vertical-align: middle;">3.9</td>
      <td align="center">11.8</td>
      <td align="center">1.8.0</td>
    </tr>
    <tr>
      <td align="center">3.10</td>
      <td align="center">12.6</td>
      <td align="center">2.7.1</td>
    </tr>
  </tbody>
</table>

* #### CUDA 버전 관련 안내사항  
  - 이번 경진대회는 3개의 CUDA 버전을 지원합니다.  
  - 참가자는 자신의 모델의 라이브러리 의존성에 맞는 CUDA 환경을 선택하여 모델을 제출하면 됩니다.   
  - 각 CUDA 환경에는 기본적으로 torch가 설치되어 있으나, 참가자는 제출하는 CUDA 버전과 호환되는 torch, 필요한 버전의 라이브러리를 `!pip install` 하여 사용하여도 무관합니다.

* #### `task.ipynb` 작성 규칙
코드는 크게 3가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하세요.   
1. **제출용 aifactory 라이브러리 및 추가 필요 라이브러리 설치**
    - 채점 및 제출을 위한 aifactory 라이브러리를 설치하는 셀입니다. 이 부분은 수정하지 않고 그대로 실행합니다.
    - 그 외로, 모델 추론에 필요한 라이브러리를 직접 설치합니다.
2. **추론용 코드 작성**
    - 모델 로드, 데이터 전처리, 예측 등 실제 추론을 수행하는 모든 코드를 이 영역에 작성합니다.
3. **aif.submit() 함수를 호출하여 최종 결과를 제출**
    - **마이 페이지-활동히스토리**에서 발급받은 key 값을 함수의 인자로 정확히 입력해야 합니다.
    - **※주의** : 제출하고자 하는 CUDA 환경에 맞는 key를 입력하여야 합니다.

<table>
  <thead>
    <tr>
      <th align="left">Competition 이름</th>
      <th align="center">CUDA</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="left">딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회</td>
      <td align="center">11.8</td>
    </tr>
    <tr>
      <td align="left">딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회 CUDA 12.6</td>
      <td align="center">12.6</td>
    </tr>
    <tr>
      <td align="left">딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회 CUDA 10.2</td>
      <td align="center">10.2</td>
    </tr>
  </tbody>
</table>

------

#### 1. 제출용 aifactory 라이브러리 설치
※ 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

In [5]:
!pip install -U aifactory

* 자신의 모델 추론 실행에 필요한 추가 라이브러리 설치

In [ ]:
!pip install torch==2.7.1 torchvision==0.22.1 --index-url https://download.pytorch.org/whl/cu126
!pip install transformers==4.30.0 datasets==2.19.1
!pip install numpy==1.26.4 scipy==1.11.4 scikit-learn==1.3.2
!pip install pandas Pillow
!pip install opencv-python-headless==4.10.0.82
!pip install dlib --no-cache-dir
!pip install pytorchvideo

---

##### CPU 전용 테스트용

In [9]:
# !pip install torch==2.7.1 torchvision==0.22.1 torchaudio --index-url https://download.pytorch.org/whl/cpu
# !pip install transformers==4.30.0 datasets==2.19.1
# !pip install numpy==1.26.4 scipy==1.11.4 scikit-learn==1.3.2
# !pip install pandas Pillow
# !pip install opencv-python-headless==4.10.0.82
# !pip install dlib --no-cache-dir


-----

#### 2. 추론용 코드 작성

##### 추론 환경의 기본 경로 구조

- 평가 데이터셋 경로: `./data/`
   - 채점에 사용될 테스트 데이터셋은 `./data/` 디렉토리 안에 포함되어 있습니다.
   - 해당 디렉토리에는 이미지(JPG, PNG)와 동영상(MP4) 파일이 별도의 하위 폴더 없이 혼합되어 있습니다.
```bash
/aif/
└── data/
    ├── {이미지 데이터1}.jpg
    ├── {이미지 데이터2}.png
    ├── {동영상 데이터1}.mp4
    ├── {이미지 데이터3}.png
    ├── {동영상 데이터2}.mp4
    ...
```

- 모델 및 자원 경로: 예시 : `./model/`
   - 추론 스크립트가 실행되는 위치를 기준으로, 제출된 모델 관련 파일들이 위치해야하 하는 상대 경로입니다.
   - 학습된 모델 가중치(.pt, .ckpt, .pth 등)

* 제출 파일은 `submission.csv`로 저장돼야 합니다.
  * submission.csv는 *filename*과 *label* 컬럼으로 구성돼야 합니다.
  * filename은 추론한 파일의 이름(확장자 포함), label은 추론 결과입니다. (real:0, fake:1)
  * filename은 *string*, label은 *int* 자료형이어야 합니다.
  * 추론하는 데이터의 순서는 무작위로 섞여도 상관 없습니다.

<table>
  <thead>
    <tr>
      <th align="center">filename</th>
      <th align="center">label</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="center">{이미지 데이터1}.jpg</td>
      <td align="center">0</td>
    </tr>
    <tr>
      <td align="center">{동영상 데이터1}.mp4</td>
      <td align="center">1</td>
    </tr>
    <tr>
      <td colspan="2" align="center">...</td>
    </tr>
  </tbody>
</table>

**※ 주의 사항**

* argparse 사용시 `args, _ = parser.parse_known_args()`로 인자를 지정하세요.   
   - `args = parser.parse_args()`는 jupyter에서 오류가 발생합니다.
* return 할 결과물과 양식에 유의하세요.

In [ ]:
import os
from pathlib import Path
import csv
import cv2
import numpy as np
from PIL import Image

import torch
import torch.nn.functional as F
from torchvision import transforms
from tqdm import tqdm

from model.scse.scse_model import ScSEModel
from model.timesformer.timeSformer_model import TimeSformerModel

# =============================
# 경로 설정
# =============================
SCSE_MODEL_PATH = "./model/weights/scse_best.pth"
TS_MODEL_PATH = "./model/weights/timeSformer_best.pth"
TEST_DIR = Path("./data")   # 테스트셋 디렉토리
SUBMISSION_PATH = Path("submission.csv")

IMAGE_EXTS = {".jpg", ".jpeg", ".png"}
VIDEO_EXTS = {".mp4", ".avi", ".mov", ".mkv"}

# =============================
# 모델 설정
# =============================
IMG_SIZE = 224
NUM_FRAMES = 8
NUM_CLASSES = 2  # real=0, fake=1 (절대 뒤집지 않음)

LABEL_FLIP = False  # 사용 안 함
VIDEO_TSF_ALPHA = 0.6  # 비디오에서 TimeSformer 비율

# 디바이스
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

# =============================
# Transform
# =============================
inference_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# =============================
# 비디오 샘플링
# =============================
def sample_frame_indices(total_frames, num_samples):
    """학습 시 VideoDataset의 np.linspace 샘플링과 동일."""
    if total_frames <= 0:
        return np.zeros(num_samples, dtype=int)

    if total_frames < num_samples:
        base = np.linspace(0, total_frames - 1, total_frames).astype(int)
        pad = np.full(num_samples - total_frames,
                      total_frames - 1, dtype=int)
        return np.concatenate([base, pad])

    return np.linspace(0, total_frames - 1, num_samples).astype(int)

# =============================
# 이미지/비디오 로더
# =============================
def load_image_file(path: Path):
    img = Image.open(str(path)).convert("RGB")
    tensor = inference_transform(img)
    return tensor.unsqueeze(0)  # (1,3,H,W)

def load_video_frames(path: Path, num_frames: int):
    cap = cv2.VideoCapture(str(path))
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    idxs = sample_frame_indices(total, num_frames)

    frames = []
    for i in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(i))
        ret, frame = cap.read()

        if not ret:
            frame = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
        else:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        img = Image.fromarray(frame)
        frames.append(inference_transform(img))  # (3,H,W)

    cap.release()
    return torch.stack(frames, dim=0)  # (T,3,H,W)

# =============================
# 모델 로딩
# =============================
# scSE
scse_model = ScSEModel(num_classes=NUM_CLASSES)
scse_model.load_state_dict(torch.load(
    SCSE_MODEL_PATH, map_location=device), strict=False)
scse_model = scse_model.to(device).eval()

# TimeSformer
ts_model = TimeSformerModel(
    num_classes=NUM_CLASSES,
    num_frames=NUM_FRAMES,
    img_size=IMG_SIZE
)
ts_model.load_state_dict(torch.load(
    TS_MODEL_PATH, map_location=device), strict=False)
ts_model = ts_model.to(device).eval()

print("모델 로드 완료!")

# =============================
# 추론 함수
# =============================
def infer_image(path: Path) -> int:
    x = load_image_file(path).to(device)

    with torch.no_grad():
        logits = scse_model(x)
        prob = F.softmax(logits, dim=1)
        pred = prob.argmax(dim=1).item()

    return int(pred)

def infer_video(path: Path) -> int:
    frames = load_video_frames(path, NUM_FRAMES)

    ts_input = frames.unsqueeze(0).to(device)   # (1,T,3,H,W)
    scse_input = frames.to(device)             # (T,3,H,W)

    with torch.no_grad():

        # TimeSformer
        if device.type == "cuda":
            with torch.cuda.amp.autocast():
                ts_logits = ts_model(ts_input)
        else:
            ts_logits = ts_model(ts_input)

        ts_prob = F.softmax(ts_logits, dim=1)  # (1,2)

        # scSE
        scse_logits = scse_model(scse_input)  # (T,2)
        scse_prob = F.softmax(scse_logits, dim=1).mean(dim=0, keepdim=True)

        # 앙상블
        final_prob = VIDEO_TSF_ALPHA * ts_prob + (1 - VIDEO_TSF_ALPHA) * scse_prob

        pred = final_prob.argmax(dim=1).item()

    return int(pred)

# =============================
# 전체 파일 추론 & CSV 저장
# =============================
def run_inference():
    files = sorted([p for p in TEST_DIR.iterdir() if p.is_file()])

    print("테스트 샘플 개수:", len(files))
    results = {}

    for path in tqdm(files, desc="Processing"):
        ext = path.suffix.lower()

        try:
            if ext in IMAGE_EXTS:
                pred = infer_image(path)
            elif ext in VIDEO_EXTS:
                pred = infer_video(path)
            else:
                pred = 0
        except Exception as e:
            print(f"[ERROR] {path.name}: {e}")
            pred = 0

        results[path.name] = pred

    # CSV 저장
    with open(SUBMISSION_PATH, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["filename", "label"])

        for p in files:
            writer.writerow([p.name, results[p.name]])

    print("submission.csv 저장 완료!")
    print(f"저장 위치: {SUBMISSION_PATH}")

run_inference()


사용 장치: cpu
모델 로드 완료!
테스트 샘플 개수: 12


Processing:  58%|█████▊    | 7/12 [00:00<00:00, 14.37it/s]

[ERROR] sample_video_1.mp4: Given groups=1, weight of size [24, 3, 1, 3, 3], expected input[1, 16, 3, 224, 224] to have 3 channels, but got 16 channels instead


Processing:  75%|███████▌  | 9/12 [00:03<00:01,  1.71it/s]

[ERROR] sample_video_2.mp4: Given groups=1, weight of size [24, 3, 1, 3, 3], expected input[1, 16, 3, 224, 224] to have 3 channels, but got 16 channels instead


...

[ERROR] sample_video_3.mp4: Given groups=1, weight of size [24, 3, 1, 3, 3], expected input[1, 16, 3, 224, 224] to have 3 channels, but got 16 channels instead


Processing:  92%|█████████▏| 11/12 [00:04<00:00,  1.73it/s]

[ERROR] sample_video_4.mp4: Given groups=1, weight of size [24, 3, 1, 3, 3], expected input[1, 16, 3, 224, 224] to have 3 channels, but got 16 channels instead


Processing: 100%|██████████| 12/12 [00:05<00:00,  2.23it/s]

[ERROR] sample_video_5.mp4: Given groups=1, weight of size [24, 3, 1, 3, 3], expected input[1, 16, 3, 224, 224] to have 3 channels, but got 16 channels instead
submission.csv 저장 완료!
저장 위치: submission.csv


----

#### 3. `aif.submit()` 함수를 호출하여 최종 결과를 제출

**※주의** : task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 대회 페이지 [베이스라인 코드](https://aifactory.space/task/9197/baseline) 탭에 기재된 가이드라인을 따라 task 별로 확인하실 수 있습니다.
- key가 틀리면 제출이 진행되지 않거나 잘못 제출되므로 task에 맞는 자신의 key를 사용해야 합니다.
-  **NOTE** : 이번 경진대회에서는 3개의 CUDA 버전을 지원하며, 각 CUDA 버전에 따라 task key가 상이합니다. 함수를 실행하기 전에 현재 key가 제출하고자 하는 CUDA 환경에 대한 key인지 반드시 확인하세요.

In [ ]:
import aifactory.score as aif
import time
t = time.time()

#-----------------------------------------------------#
aif.submit(model_name="scSE-x3ds_test",
    key="My-KEY"
)
#-----------------------------------------------------#
print(time.time() - t)

file : task
jupyter notebook
제출 완료
45.15132021903992
